In [ ]:
import cobra
from cobra.flux_analysis import flux_variability_analysis
from cobra import Model, Reaction, Metabolite

In [ ]:
DraftTmurisV3 = cobra.io.read_sbml_model("Draft_Tmuris_V3.xml")


In [ ]:

i = 0
while i < len(DraftTmurisV3.reactions)-1:
    MetAdd = []
    FVA = flux_variability_analysis(DraftTmurisV3,DraftTmurisV3.reactions[i])
    if float(FVA.maximum) == 0.0 and float(FVA.minimum) == 0.0:
        print("\t \t Reaction %s is being checked \n" % DraftTmurisV3.reactions[i].id)
        RxnMets = []
        for l in DraftTmurisV3.reactions[i].reactants:
            RxnMets.append(l.id)
            
        with DraftTmurisV3 as model:   
            for k in range(len(RxnMets)):
                if RxnMets[k][-1] == "c":
                    ExMet = str(RxnMets[k][:-1])+"e"
                    
                    try:
                        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ \n")
                        print("Metabolite already exist: %s \t %s" % (model.metabolites.get_by_id(str(ExMet)),RxnMets[k]))
                        print("\n")
                    except Exception:
                        MetAdd.append(ExMet)
                        print("\n Metabolite to be checked: %s \t %s" % (RxnMets[k],ExMet))
                        #print("\n")
                        RxnID = "New_reaction"+str(i)+"_"+str(k)
                        reaction = Reaction(RxnID)
                        reaction.name = str(RxnMets[k])+"transport"
                        reaction.lower_bound = -999999.0
                        reaction.upper_bound = 999999.0
                        ExternalMet = Metabolite(ExMet,
                                                  formula=model.metabolites.get_by_id(str(RxnMets[k])).formula,
                                                  name=model.metabolites.get_by_id(str(RxnMets[k])).name,
                                                  compartment="e")
                        reaction.add_metabolites({ExternalMet: -1.0})
                        model.add_reactions([reaction])
                
                        RxnID2 = "New_reaction"+str(i)+"_"+str(k)+"_"+"2"
                        reaction_2 = Reaction(RxnID2)
                        reaction_2.name = str(RxnMets[k])+"transport"
                        reaction_2.lower_bound = -999999.0
                        reaction_2.upper_bound = 999999.0
                        reaction_2.add_metabolites({ExternalMet: -1.0,model.metabolites.get_by_id(str(RxnMets[k])): 1.0})
                        model.add_reactions([reaction_2])
                        print("\t checking is completed and ","%s is added" % (ExMet),"\n")
                        print("-----------------------------------")
            print("\n \t Metabolite addition from -%s- is finished:" % DraftTmurisV3.reactions[i].id,"\n")            
            zero_flux = []
            t = 0
            while t < len(model.reactions):
                FVA2 = flux_variability_analysis(model,model.reactions[t])
                if float(FVA2.maximum) == 0.0 and float(FVA2.minimum) == 0.0:
                    zero_flux.append(model.reactions[t].id)
                t += 1
                
            length_of_temporary_model = len(model.reactions)
            length_of_added_metabolites = len(MetAdd)
            print("\t \tThe number of zero flux reactions after -%s- metabolite(s) from -%s- is added: \t %d" % (length_of_added_metabolites,DraftTmurisV3.reactions[i].id,len(zero_flux)))  
            print("\t \tThe total number of the temporary model reactions after new reactions are added: \t %d" % (len(model.reactions)))
            #print("\n")
            
            
        print("\t ********************************Next reaction*********************************** \n")     
        
                    
                    
  
        
   
    i += 1